In [136]:
import pandas as pd

In [161]:
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

ps = PorterStemmer()
nltk.download('stopwords')
stop = stopwords.words("english")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [138]:
df = pd.read_csv("/content/IMDB Dataset.csv", nrows=500)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [139]:
df.shape

(500, 2)

In [140]:
df["sentiment"].replace({"positive":1,"negative":0},inplace=True)

In [141]:
corpus = []
for i in range(0, 500):
    text = re.sub(r"[^\s\w]", "", df["review"][i])
    text = re.sub(r"<br\s*\/?>", "", text)
    text = text.replace(',', ' ')
    text = text.replace('\'', ' ')
    text = text.replace('"', ' ')
    text = re.sub(r"https\s+|http\s+|www\s+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(i) for i in text if i not in set(stop)]
    text = " ".join(text)
    corpus.append(text)

corpus


['one review mention watch 1 oz episod youll hook right exactli happen mebr br first thing struck oz brutal unflinch scene violenc set right word go trust not show faint heart timid show pull punch regard drug sex violenc hardcor classic use wordbr br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci not high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awaybr br would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti surreal couldnt say readi watch develop tast oz got accustom high level graphic violenc not violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison expe

In [142]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(max_features=6211)

In [143]:
x = cv.fit_transform(corpus).toarray()

In [144]:
y= df.iloc[:,-1]

In [145]:
import pickle
pickle.dump(cv,open("/content/CV_model.csv","wb"))

In [146]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [147]:
from sklearn.naive_bayes import GaussianNB

nb=GaussianNB()
nb.fit(x_train,y_train)

GaussianNB()

In [148]:
import joblib
joblib.dump(nb,"/content/Model.csv")

['/content/Model.csv']

In [149]:
from sklearn.metrics import accuracy_score
ypred = nb.predict(x_test)
accuracy_score(y_test,ypred)

0.52

In [150]:
df = pd.read_csv("/content/IMDB Dataset.csv",nrows=200)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [151]:
df["sentiment"].replace({"positive":" ","negative":" "},inplace=True)

In [152]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,
1,A wonderful little production. <br /><br />The...,
2,I thought this was a wonderful way to spend ti...,
3,Basically there's a family where a little boy ...,
4,"Petter Mattei's ""Love in the Time of Money"" is...",


In [153]:
cor=[]
for i in range(0,200):
    text = re.sub(r"[^\s\w]", "", df["review"][i])
    text = re.sub(r"<br\s*\/?>", "", text)
    text = text.replace(',', ' ')
    text = text.replace('\'', ' ')
    text = text.replace('"', ' ')
    text = re.sub(r"https\s+|http\s+|www\s+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(i) for i in text if i not in set(stop)]
    text = " ".join(text)
    cor.append(text)
cor

['one review mention watch 1 oz episod youll hook right exactli happen mebr br first thing struck oz brutal unflinch scene violenc set right word go trust not show faint heart timid show pull punch regard drug sex violenc hardcor classic use wordbr br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci not high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awaybr br would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti surreal couldnt say readi watch develop tast oz got accustom high level graphic violenc not violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison expe

In [154]:
import pickle

cv=pickle.load(open("/content/CV_model.csv","rb"))

In [155]:
x_new=cv.fit_transform(cor).toarray()

In [156]:
import joblib
nb=joblib.load("/content/Model.csv")

In [157]:
ypred=nb.predict(x_new)

In [158]:
df["sentiment"] = ypred.tolist()

In [160]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,1
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
195,Phantasm ....Class. Phantasm II.....awesome. P...,1
196,Ludicrous. Angelic 9-year-old Annakin turns in...,1
197,"Scotty (Grant Cramer, who would go on to star ...",1
198,If you keep rigid historical perspective out o...,1
